# **DISCLAIMER/AVISO**

O script a seguir é uma junção do script fornecido pela monitora Glória Carvalho da disciplina de Métodos e Técnicas de Pesquisa II, com adições e trocas a partir de sugestões do Gemini, que reformulou algumas linhas de acordo com o nosso arquivo. Portanto, que fique claro que esse script não foi escrito por nenhum membro desse grupo.

# **(1) Instalar e carregar as bibliotecas**

In [ ]:
!pip install corextopic

import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import corextopic
from corextopic import corextopic as ct

# **(2) Abrindo e lendo a planilha**

In [ ]:
df = pd.read_excel('/content/LDA_final.xlsx')

caminho = '/content/LDA_final.xlsx'

nomes_colunas = ["key", "jornal", "transcrito", "tópico 1", "tópico 2", "tópico 3", "tópico 4", "tópico 5"]

df = pd.read_excel(caminho, names=nomes_colunas)

In [ ]:
# (2.1) Exibir os resultados

df.head()

# **(3) Criando um dicionário a partir das palavras das matérias e inserindo elas em um novo dataframe**

In [ ]:
rows = []

In [ ]:
# (3.1) Itera sobre cada linha da coluna "transcrito" e adiciona o texto a uma nova lista de dicionários

for text in df['transcrito']:
    rows.append({"text": text})

In [ ]:
# (3.2) Cria um novo DataFrame a partir dessa lista de dicionários

new_df = pd.DataFrame(rows)

In [ ]:
# (3.3) Exibe o tamanho do novo DataFrame (para certificar de que todas as células textuais foram realocadas)

print(len(new_df))

In [ ]:
# (3.4) Exibe o novo DataFrame para verificar o resultado

print(new_df.head())

# **(4) Transformando as palavras em vetores**

In [ ]:
# (4.1) Cria um objeto TfidVectorizer com as especificações desejadas

vectorizer = TfidfVectorizer(
    max_df=75,
    min_df=2,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

In [ ]:
# (4.2) Ajusta o vetorizador aos textos da coluna 'text'

vectorizer = vectorizer.fit(new_df['text'])

In [ ]:
# (4.3) Transforma os textos em uma matriz TF-IDF

tfidf = vectorizer.transform(new_df['text'])

In [ ]:
# (4.4) Obtém o vocabulário utilizado no modelo TF-IDF

vocab = vectorizer.get_feature_names_out()

In [ ]:
# (4.5) Exibe o tamanho do vocabulário

print(len(vocab))

# **(5) Criando as palavras âncoras**

In [ ]:
anchors = []
model = ct.Corex(n_hidden=4, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

# **(6) Treinando o modelo sem supervisão (modelagem não supervisionada)**

In [ ]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

# **(7) Fazendo a modelagem semi-supervisionada (usando as palavras âncoras)**

*Modelagem apenas do Estadão*

In [ ]:
# (7.1) Filtrando o DataFrame para incluir apenas textos do Estadão

df_estadao = df[df['jornal'] == 'Estadão']

# (7.2) Repetir os procedimentos do (3.1) até o (4.5)

rows = []

for text in df_estadao['transcrito']:
    rows.append({"text": text})

new_df = pd.DataFrame(rows)

print(len(new_df))

print(new_df.head())

vectorizer = TfidfVectorizer(
    max_df=75,
    min_df=2, # Changed min_df from 0.5 to 2
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

vectorizer = vectorizer.fit(new_df['text'])
tfidf = vectorizer.transform(new_df['text'])
vocab = vectorizer.get_feature_names_out()

print(len(vocab))

# (7.3) Criando as palavras âncoras e direcionando o algoritmo

anchors = [
    ["greve"],
    ["decreto"],
    ["policia"],
    ["jose_serra"],
     ["reitoria"]]

anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=5, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors,
    anchor_strength=3
)

# (7.4) Roda a modelagem utilizando as palavras âncoras estipuladas

for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))


*Modelagem apenas da Folha de São Paulo*

In [ ]:
# (7.5) Repetir o mesmo processo, mas agora para a FSP

df_fsp = df[df['jornal'] == 'FSP']

rows = []
for text in df_fsp['transcrito']:
    rows.append({"text": text})

new_df = pd.DataFrame(rows)

print(len(new_df))

print(new_df.head())

vectorizer = TfidfVectorizer(
    max_df=75,
    min_df=2,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

vectorizer = vectorizer.fit(new_df['text'])

tfidf = vectorizer.transform(new_df['text'])

vocab = vectorizer.get_feature_names_out()

print(len(vocab))

anchors = [
    ["reitoria"],
    ["greve"],
    ["decreto"],
    ["policia"],
    ["jose_serra"]]

anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=5, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors,
    anchor_strength=3
)

for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))


*Modelagem apenas do Jornal da USP*

In [ ]:
# (7.6) Repetir o mesmo processo, mas agora para o Jornal da USP

df_Jornal_USP = df[df['jornal'] == 'Jornal_USP']

rows = []

for text in df_Jornal_USP['transcrito']:
    rows.append({"text": text})

new_df = pd.DataFrame(rows)

print(len(new_df))

print(new_df.head())

vectorizer = TfidfVectorizer(
    max_df=75,
    min_df=2,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

vectorizer = vectorizer.fit(new_df['text'])

tfidf = vectorizer.transform(new_df['text'])

vocab = vectorizer.get_feature_names_out()

print(len(vocab))

anchors = [
    ["reitoria"],
    ["decreto"],
    ["congregacao"],
    ["estudantes"],
    ["patrimonio"]]

anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=5, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors,
    anchor_strength=3
)

for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))


*Modelagem apenas do Jornal do Campus*

In [ ]:
# (7.7) Repetir o mesmo processo, mas agora para o Jornal do Campus
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import corextopic
from corextopic import corextopic as ct

df_JC = df[df['jornal'] == 'JC']

rows = []

for text in df_JC['transcrito']:
    rows.append({"text": text})

new_df = pd.DataFrame(rows)

print(len(new_df))

print(new_df.head())

vectorizer = TfidfVectorizer(
    max_df=75,
    min_df=2,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

vectorizer = vectorizer.fit(new_df['text'])

tfidf = vectorizer.transform(new_df['text'])

vocab = vectorizer.get_feature_names_out()

print(len(vocab))

anchors = [
    ["reitoria"],
    ["autonomia"],
    ["estudante"],
    ["sintusp"],
    ["adusp"],]

anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=5, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors,
    anchor_strength=3
)

for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))